# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
## Creating list of filepaths to process original event csv data files                                                
                                                                                                                      
# check and print your current working directory                                                                      
print(os.getcwd())                                                                                                    
                                                                                                                      
# Get your current folder and subfolder event data                                                                    
filepath = os.getcwd() + '/event_data'                                                                                
                                                                                                                      
# Create a for loop to create a list of files and collect each filepath                                               
for root, dirs, files in os.walk(filepath):                                                                           
                                                                                                                      
    # join the file path and roots with the subdirectories using glob                                                 
    file_path_list = glob.glob(os.path.join(root,'*'))                                                                
    #print(file_path_list)                                                                                            

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
## Process CSV files to create the final data file                                                                     
                                                                                                                       
# initiating an empty list of rows that will be generated from each file                                               
full_data_rows_list = []                                                                                               
                                                                                                                       
# for every filepath in the file path list                                                                             
for f in file_path_list:                                                                                               
                                                                                                                       
    # reading csv file                                                                                                 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:                                                       
        # creating a csv reader object                                                                                 
        csvreader = csv.reader(csvfile)                                                                                
        next(csvreader)                                                                                                
                                                                                                                       
        # extracting each data row one by one and append it                                                            
        for line in csvreader:                                                                                         
            #print(line)                                                                                               
            full_data_rows_list.append(line)                                                                           
            
# Generate final CSV data file                                                                                          
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)                                         
                                                                                                                        
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:                                           
    writer = csv.writer(f, dialect='myDialect')                                                                         
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',                                
                     'level','location','sessionId','song','userId'])                                                   
    for row in full_data_rows_list:                                                                                     
        if (row[0] == ''):                                                                                              
            continue                                                                                                    
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))    


In [4]:
# check the number of rows in your csv file                                                                             
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:                                                       
    print(sum(1 for line in f))                                                                                         

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# connect to Cassandra and create Keyspace                       
from cassandra.cluster import Cluster                            

cluster = Cluster()

# obtain session  
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create the Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

#### Set Keyspace

In [7]:
# Set the KEYSPACE 
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Task 1: Getting the artist, song's title and song's length for a given session_id and item_in_session

query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length double, " \
                "PRIMARY KEY (session_id, item_in_session))"

session.execute(query)

In [9]:
## Process new data file                                                                                                
file = 'event_datafile_new.csv'                                                                                         
                                                                                                                        
with open(file, encoding='utf8') as f:                                                                                  
    csvreader = csv.reader(f)                                                                                           
    next(csvreader)  # skip header                                                                                      
    for line in csvreader:                                                                                              
        ## Define INSERT statement                                                                                      
        query = "INSERT INTO song_library (session_id, item_in_session, artist_name, song_title, song_length)"          
        query = query + "VALUES (%s, %s, %s, %s, %s)"                                                                   
        ## Run INSERT statement                                                                                         
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))                          
                                                                                                                        
## Execute SELECT to validate the output                                                                                
query = "SELECT artist_name, song_title, song_length FROM song_library WHERE session_id=411 AND item_in_session = 4"

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Execute SELECT to validate the output                                                                                
query = "SELECT artist_name, song_title, song_length FROM song_library WHERE session_id=411 AND item_in_session = 4"   
rows = session.execute(query)                                                                                           
                                                                                                                        
for row in rows:                                                                                                        
    print (row)                                                                                                         

Row(artist_name='OneRepublic', song_title='Secrets', song_length=224.67873)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Task 2: Getting the artist, song's title and user for a given user_id and session_id, sorting by item_in_session     
query = "CREATE TABLE IF NOT EXISTS user_play "                                                                         
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, " \
                "first_name text, last_name text, PRIMARY KEY ((user_id, session_id), item_in_session))"              
session.execute(query)                                                                                                  
                                                                                                                        
file = 'event_datafile_new.csv'                                                                                         
                                                                                                                        
with open(file, encoding = 'utf8') as f:                                                                                
    csvreader = csv.reader(f)                                                                                           
    next(csvreader) # skip header                                                                                       
    for line in csvreader:                                                                                              
        ## Define INSERT statement                                                                                      
        query = "INSERT INTO user_play (user_id, session_id, item_in_session, artist_name, song_title, ";               
        query = query + "first_name, last_name)";                                                                       
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"                                                              
        ## Run INSERT statement                                                                                         
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))         
                                                                                                                        
## Execute SELECT to validate the output                                                                                
query = "SELECT artist_name, song_title, first_name, last_name FROM user_play WHERE user_id=10 AND session_id=182"      
rows = session.execute(query)                                                                                           
                                                                                                                        
for row in rows:                                                                                                        
    print (row)                                                                                                         

Row(artist_name='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist_name='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist_name='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


In [12]:
## Task 3: Getting the users who listened to a given song                                                               
query = "CREATE TABLE IF NOT EXISTS song_history "                                                                      
query = query + "(song_title text, first_name text, last_name text, user_id int, " \
            "PRIMARY KEY (song_title, first_name, last_name, user_id))"                                             
session.execute(query)                                                                                                  
                                                                                                                        
file = 'event_datafile_new.csv'                                                                                         
                                                                                                                        
with open(file, encoding = 'utf8') as f:                                                                                
    csvreader = csv.reader(f)                                                                                           
    next(csvreader) # skip header                                                                                       
    for line in csvreader:                                                                                              
        ## Define INSERT statement                                                                                      
        query = "INSERT INTO song_history (song_title, first_name, last_name, user_id)"                                 
        query = query + "VALUES (%s, %s, %s, %s)"                                                                       
        ## Run INSERT statement                                                                                         
        session.execute(query, (line[9], line[1], line[4], int(line[10])))                                              
                                                                                                                        
## Execute SELECT to validate the output                                                                                
query = "SELECT first_name, last_name FROM song_history WHERE song_title='All Hands Against His Own'"                   
rows = session.execute(query)                                                                                           
                                                                                                                        
for row in rows:                                                                                                        
    print (row)                                                                                                         

SyntaxError: unexpected character after line continuation character (<ipython-input-12-b5c68dec50b8>, line 3)

### Drop the tables before closing out the sessions

In [ ]:
## Remove tables 

query = "DROP TABLE song_library"
session.execute(query)

query = "DROP TABLE user_play"
session.execute(query)

query = "DROP TABLE song_history"
session.execute(query)


### Close the session and cluster connection¶

In [ ]:
## Shutdown resources 

session.shutdown()
cluster.shutdown()